In [1]:
import glob 
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import random
import time
import threading
import queue
import numpy as np

In [ ]:
def process_data(queue_data, output_folder):
    while True:
        try:
            # Wait for 1 second for data to be available in the queue
            data = queue_data.get(timeout=1.0)  

            # Create a filename using the timestamp of the data
            timestamp = str(int(time.time() * 1000))  # Milliseconds since epoch
            filename = os.path.join(output_folder, f'data_{timestamp}.txt')

            # Write the data to the file
            with open(filename, 'w') as f:
                f.write(str(data))

        except queue.Empty:
            continue

In [2]:
output_folder = '/home/parsa/Cheetah/carla_simulator/out'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()
blueprint_library = world.get_blueprint_library()

In [4]:
# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

2556

In [5]:
actor_list = []
# Spawn a vehicle of random choice into world
bp = random.choice(blueprint_library.filter('vehicle'))
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(bp, transform)
actor_list.append(vehicle)
vehicle.set_autopilot(True)
print(f'created {vehicle.type_id}')

created vehicle.volkswagen.t2


In [6]:
# Spawn a camera attached to the vehicle
camera_rgb = world.spawn_actor(
    blueprint_library.find('sensor.camera.rgb'),
    carla.Transform(carla.Location(x=1.5, z=2.4)),
    attach_to=vehicle)
actor_list.append(camera_rgb)
print(f'created {camera_rgb.type_id}')

created sensor.camera.rgb


In [ ]:
# Set up the data processing thread
data_queue = queue.Queue()
data_thread = threading.Thread(target=process_data, args=(data_queue, output_folder))
data_thread.start()

In [8]:
# Spawn a vehicle of random choice into world
bp = random.choice(blueprint_library.filter('vehicle'))
vehicle2 = world.spawn_actor(bp, random.choice(world.get_map().get_spawn_points()))
actor_list.append(vehicle2)
vehicle2.set_autopilot(True)
print(f'created {vehicle2.type_id}')

created vehicle.micro.microlino


In [ ]:
record_time = 60 # Seconds
time0 = time.time()
info = []
info2 = []
while True:
    # Synchronize the simulation
    world.tick()

    # Get data from the sensors
    info.append(vehicle.get_location())
    info.append(vehicle.get_velocity())
    info.append(vehicle.get_acceleration())

    info2.append(vehicle2.get_location())
    info2.append(vehicle2.get_velocity())
    info2.append(vehicle2.get_acceleration())
    
    # # Put the data into the queue
    # data = {
    #     'vehicle_location': (vehicle_location.x,vehicle_location.y,vehicle_location.z),
    #     'vehicle_velocity': (vehicle_velocity.x,vehicle_velocity.y,vehicle_velocity.z),
    #     'vehicle_acceleration': (vehicle_acceleration.x,vehicle_acceleration.y,vehicle_acceleration.z),
    # }
    # data_queue.put(data)

    time1 = time.time()
    if (time1-time0) > record_time:
        break
    
time.sleep(5)

In [ ]:
print("Destroying Actors")
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])
data_thread.join()
print("Done.")

In [22]:
print(info[3001])

Vector3D(x=-0.017404, y=0.013099, z=-0.000695)


In [32]:
print(info2[3001])

Vector3D(x=-0.006050, y=-0.000545, z=0.004901)


In [34]:
x = info[0]
x2 = info2[0]

In [39]:
x.distance(x2)

135.0155487060547

: 

In [37]:
((x.x - x2.x)**2 + (x.y - x2.y)**2 + (x.z - x2.z)**2)**0.5

135.01555324632878